In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [122]:
top_80s = pd.read_csv('top_100_80s.csv')
top_80s = top_80s.drop(columns=['target'])
top_80s

,Song,Artist Names,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections
0,Physical,Olivia Newton-John,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,0.0631,0.000050,0.3130,0.904,123.815,224773,4,46.39820,9
1,Bette Davis Eyes,Kim Carnes,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,0.0204,0.000000,0.0959,0.596,116.624,228000,4,42.02573,9
2,Endless Love,Diana Ross & Lionel Richie,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,0.6650,0.000248,0.1380,0.237,93.633,266267,4,38.48246,11
3,Eye Of The Tiger,Survivor,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,0.1320,0.000311,0.0873,0.548,108.873,245640,4,26.24707,12
4,Every Breath You Take,The Police,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,0.5430,0.002940,0.0714,0.740,117.401,253920,4,69.63651,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Don't Know Much,Linda Ronstadt (Featuring Aaron Neville),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,0.6570,0.000011,0.1580,0.267,130.465,211973,4,50.90012,9
96,Got My Mind Set On You,George Harrison,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,0.4650,0.000002,0.4850,0.963,149.206,234653,4,18.89341,13
97,9 To 5,Dolly Parton,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,0.4160,0.000002,0.6310,0.813,105.390,162467,4,25.12092,7
98,Rock Me Amadeus,Falco,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,0.2960,0.003360,0.0884,0.889,176.737,202236,4,19.08299,10


In [123]:
top_80s['Song'] = top_80s['Song'].str.replace('\"', '')
top_80s['Song'] = top_80s['Song'].str.upper()
top_80s = top_80s.rename(columns={'Song': 'song'})

top_80s.rename(columns={'Artist Names': 'artist'}, inplace=True)
top_80s['artist'] = top_80s['artist'].str.replace('\"', '')
top_80s['artist'] = top_80s['artist'].str.upper()
top_80s

,song,artist,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections
0,PHYSICAL,OLIVIA NEWTON-JOHN,1,spotify:track:3hVrI7snZMTmHCnbwME47u,0.834,0.633,4,-11.442,0,0.0422,0.0631,0.000050,0.3130,0.904,123.815,224773,4,46.39820,9
1,BETTE DAVIS EYES,KIM CARNES,2,spotify:track:0odIT9B9BvOCnXfS0e4lB5,0.675,0.649,5,-10.271,1,0.0319,0.0204,0.000000,0.0959,0.596,116.624,228000,4,42.02573,9
2,ENDLESS LOVE,DIANA ROSS & LIONEL RICHIE,3,spotify:track:0FGxAEMIE5GhyFqFMHRWaO,0.456,0.332,10,-9.775,1,0.0250,0.6650,0.000248,0.1380,0.237,93.633,266267,4,38.48246,11
3,EYE OF THE TIGER,SURVIVOR,4,spotify:track:2KH16WveTQWT6KOG9Rg6e2,0.817,0.599,0,-9.249,0,0.0328,0.1320,0.000311,0.0873,0.548,108.873,245640,4,26.24707,12
4,EVERY BREATH YOU TAKE,THE POLICE,5,spotify:track:1JSTJqkT5qHq8MDJnJbRE1,0.820,0.452,1,-9.796,1,0.0348,0.5430,0.002940,0.0714,0.740,117.401,253920,4,69.63651,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,DON'T KNOW MUCH,LINDA RONSTADT (FEATURING AARON NEVILLE),96,spotify:track:5z3ZDMP02xF33yCvPFnct3,0.401,0.285,11,-11.005,1,0.0327,0.6570,0.000011,0.1580,0.267,130.465,211973,4,50.90012,9
96,GOT MY MIND SET ON YOU,GEORGE HARRISON,97,spotify:track:3OeUlriM0EZHdWleJtjoVr,0.767,0.854,4,-6.947,1,0.0426,0.4650,0.000002,0.4850,0.963,149.206,234653,4,18.89341,13
97,9 TO 5,DOLLY PARTON,98,spotify:track:4w3tQBXhn5345eUXDGBWZG,0.554,0.783,6,-5.852,1,0.0457,0.4160,0.000002,0.6310,0.813,105.390,162467,4,25.12092,7
98,ROCK ME AMADEUS,FALCO,99,spotify:track:0DfG1ltJnZyq4Tx3ZLL7ZU,0.610,0.817,4,-9.450,0,0.1330,0.2960,0.003360,0.0884,0.889,176.737,202236,4,19.08299,10


In [120]:
top_90s = pd.read_csv('top_100_90s.csv')
top_90s

,Song,Artist Names,Hot100 Rank,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Smooth,Santana Featuring Rob Thomas,1,spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,0.923,9,-3.908,1,0.0338,0.160000,0.000005,0.2950,0.961,115.996,294987,4,44.23530,11,1
1,How Do I Live,LeAnn Rimes,2,spotify:track:7BD50ATrF3Vab5FQy7vtK8,0.577,0.462,2,-7.989,1,0.0279,0.128000,0.000000,0.0822,0.258,128.303,266973,4,43.73251,11,1
2,Macarena (Bayside Boys Mix),Los Del Rio,3,spotify:track:4oaj36KzXRgDg4McgcTsZK,0.927,0.721,8,-12.561,1,0.0908,0.422000,0.000044,0.0422,0.965,103.018,249382,4,25.74047,15,1
3,Un-Break My Heart,Toni Braxton,4,spotify:track:5Ihd9HrPvOADyVoonH9ZjB,0.625,0.426,2,-8.392,0,0.0271,0.413000,0.000000,0.1160,0.112,110.018,270333,4,53.40658,9,1
4,Down So Long,Jewel,5,spotify:track:5WmOYQAMavY2nk7tktsPbZ,0.611,0.550,9,-8.611,0,0.0436,0.306000,0.006590,0.1120,0.622,78.501,255147,4,36.83087,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I Need To Know,Marc Anthony,96,spotify:track:7ffwRz8lZyDOE4Vj58Lo72,0.813,0.949,3,-2.563,0,0.0336,0.486000,0.000010,0.1120,0.792,115.061,227707,4,31.46063,10,1
96,Jump,Kris Kross,97,spotify:track:27AHAtAirQapVldIm4c9ZX,0.749,0.745,2,-10.625,1,0.1640,0.000501,0.004030,0.3330,0.481,101.972,195107,4,47.80584,10,1
97,Nobody,Keith Sweat Featuring Athena Cage,98,spotify:track:4SF1747p541umnykBp352Q,0.699,0.492,9,-9.449,0,0.0318,0.484000,0.000000,0.0482,0.388,117.907,264773,4,32.47790,12,1
98,Back At One,Brian McKnight,99,spotify:track:6mwA6YiKDjAUG8kWvRRUPh,0.661,0.346,11,-9.115,1,0.0300,0.452000,0.000000,0.1310,0.256,129.752,263667,4,40.51017,7,1


In [89]:
top_2017_2021 = pd.read_csv('top_2017_to_2021.csv')
top_2017_2021.rename(columns={'title': 'song'}, inplace=True)
top_2017_2021['year'] = top_2017_2021['date'].str.split('-').str[0].astype(int)
top_2017_2021 = top_2017_2021[['song', 'artist', 'year', 'streams', 'url']]
top_2017_2021 = top_2017_2021.fillna(0)
top_2017_2021

,song,artist,year,streams,url
0,Chantaje (feat. Maluma),Shakira,2017,253019.0,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
1,Vente Pa' Ca (feat. Maluma),Ricky Martin,2017,223988.0,https://open.spotify.com/track/7DM4BPaS7uofFul...
2,Reggaetón Lento (Bailemos),CNCO,2017,210943.0,https://open.spotify.com/track/3AEZUABDXNtecAO...
3,Safari,"J Balvin, Pharrell Williams, BIA, Sky",2017,173865.0,https://open.spotify.com/track/6rQSrBHf7HlZjtc...
4,Shaky Shaky,Daddy Yankee,2017,153956.0,https://open.spotify.com/track/58IL315gMSTD37D...
...,...,...,...,...,...
26173509,BYE,Jaden,2021,0.0,https://open.spotify.com/track/3OUyyDN7EZrL7i0...
26173510,Pillars,My Anh,2021,0.0,https://open.spotify.com/track/6eky30oFiQbHUAT...
26173511,Gái Độc Thân,Tlinh,2021,0.0,https://open.spotify.com/track/2klsSb2iTfgDh95...
26173512,Renegade (feat. Taylor Swift),Big Red Machine,2021,0.0,https://open.spotify.com/track/1aU1wpYBSpP0M6I...


In [91]:
top_2017_2021.isna().sum()

song       0
artist     0
year       0
streams    0
url        0
dtype: int64

In [92]:
streaming_2021 = top_2017_2021[top_2017_2021['year'] == 2021]

streaming_2021.loc[:,'song'] = streaming_2021.loc[:,'song'].str.replace('\"', '')
streaming_2021.loc[:,'song'] = streaming_2021.loc[:,'song'].str.upper()

streaming_2021.loc[:,'artist'] = streaming_2021.loc[:,'artist'].str.replace('\"', '')
streaming_2021.loc[:,'artist'] = streaming_2021.loc[:,'artist'].str.upper()
streaming_2021

,song,artist,year,streams,url
63293,STAY GOLD,BTS,2021,54393.0,https://open.spotify.com/track/3Ys2PYl1wyPKQIw...
69327,CLOSE EYES,DVRST,2021,0.0,https://open.spotify.com/track/3CLSHJv5aUROAN2...
71618,PARÍS,INGRATAX,2021,0.0,https://open.spotify.com/track/4bF2y8rR7GUjrd5...
71725,DUVET,BÔA,2021,0.0,https://open.spotify.com/track/7urtwZEtOypb3r8...
478448,POBLADO - REMIX,"J BALVIN, KAROL G, NICKY JAM, CRISSIN, TOTOY E...",2021,69591.0,https://open.spotify.com/track/2K1CQMoG2Dy5nme...
...,...,...,...,...,...
26173509,BYE,JADEN,2021,0.0,https://open.spotify.com/track/3OUyyDN7EZrL7i0...
26173510,PILLARS,MY ANH,2021,0.0,https://open.spotify.com/track/6eky30oFiQbHUAT...
26173511,GÁI ĐỘC THÂN,TLINH,2021,0.0,https://open.spotify.com/track/2klsSb2iTfgDh95...
26173512,RENEGADE (FEAT. TAYLOR SWIFT),BIG RED MACHINE,2021,0.0,https://open.spotify.com/track/1aU1wpYBSpP0M6I...


In [93]:
sum_streaming_2021 = streaming_2021.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2021


,song,artist,year,streams,url
0,!,"SAMEY, GLEB",2021,3581.0,https://open.spotify.com/track/1A05ibu1DXGIt0F...
1,!!!,"NNS, KKN, RYU",2021,0.0,https://open.spotify.com/track/50F2cUDhrqVGoei...
2,#1090 ~THOUSAND DREAMS~,松本孝弘,2021,0.0,https://open.spotify.com/track/0Bh4YOM4VCSY0BO...
3,"#1TAKE, PT. 2","P110, MIST",2021,0.0,https://open.spotify.com/track/24bNtsddXyIJwWK...
4,#AICHOAI,"FLO D, GIANG NGUYỄN",2021,111969.0,https://open.spotify.com/track/5xOkL4eAs735nm6...
...,...,...,...,...,...
44409,지나오다,NILO,2021,0.0,https://open.spotify.com/track/0keu6b483OyisjQ...
44410,친구,BTS,2021,356265.0,https://open.spotify.com/track/2dkFOWqyiVFS3xs...
44411,키스는 다음에,LOONA,2021,93903.0,https://open.spotify.com/track/3DGKXLtVjawKFwe...
44412,항상 (HANGSANG),"J-HOPE, SUPREME BOI",2021,0.0,https://open.spotify.com/track/4mYu3kfBCW6qiTD...


In [97]:
streaming_2020 = top_2017_2021[top_2017_2021['year'] == 2020]

streaming_2020.loc[:,'song'] = streaming_2020.loc[:,'song'].str.replace('\"', '')
streaming_2020.loc[:,'song'] = streaming_2020.loc[:,'song'].str.upper()

streaming_2020.loc[:,'artist'] = streaming_2020.loc[:,'artist'].str.replace('\"', '')
streaming_2020.loc[:,'artist'] = streaming_2020.loc[:,'artist'].str.upper()
streaming_2020

,song,artist,year,streams,url
19128,RIDE IT,REGARD,2020,1956.0,https://open.spotify.com/track/2tnVG71enUj33Ic...
19234,+LINDA,DALEX,2020,1953.0,https://open.spotify.com/track/5cBrOhKDyiJF9bP...
19287,WOAH,LIL BABY,2020,35211.0,https://open.spotify.com/track/02RCbjb9czvQKNG...
19390,25/8,BAD BUNNY,2020,31854.0,https://open.spotify.com/track/4NqQaovM14WR2hN...
19698,KEII,ANUEL AA,2020,16155.0,https://open.spotify.com/track/5eEQCfq7MXQr6aH...
...,...,...,...,...,...
21437959,MAD AT DISNEY,SALEM ILESE,2020,0.0,https://open.spotify.com/track/7aGyRfJWtLqgJaZ...
21437960,EM ĐÃ THƯƠNG NGƯỜI TA HƠN ANH,NOO PHƯỚC THỊNH,2020,0.0,https://open.spotify.com/track/39t3ly1KisXk4u5...
21437961,3 1 0 7,"W/N, DUONG, NÂU",2020,0.0,https://open.spotify.com/track/42pZsfo15M63tgC...
21437962,MISSING YOU,"PHUONG LY, TINLE",2020,0.0,https://open.spotify.com/track/1AAGvzJRWMTKx7P...


In [98]:
sum_streaming_2020 = streaming_2020.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2020

,song,artist,year,streams,url
0,!,"SAMEY, GLEB",2020,133718.0,https://open.spotify.com/track/1A05ibu1DXGIt0F...
1,#%_! DIG SELV,ARTIGEARDIT,2020,85639.0,https://open.spotify.com/track/6Js5Mxd61ze79K8...
2,#213,YL,2020,49321.0,https://open.spotify.com/track/1FdmvW7UjsG7Uiu...
3,#24,DOLU KADEHI TERS TUT,2020,649313.0,https://open.spotify.com/track/6ihcJlZFxGvqIq7...
4,#24,"PROFJAM, BENJI PRICE",2020,131173.0,https://open.spotify.com/track/0wlOyxuja1SEoqE...
...,...,...,...,...,...
47885,어머님이 누구니 (WHO'S YOUR MAMA?) (FEAT. JESSI),J.Y. PARK,2020,0.0,https://open.spotify.com/track/7Hjmz2XTMCuFvKB...
47886,우린 친구뿐일까,SONDIA,2020,6075.0,https://open.spotify.com/track/0w27Ok6tBj1QBNq...
47887,지코 아무노래,HAMAH MUSIC,2020,20783.0,https://open.spotify.com/track/7CAigOoiIydhI1G...
47888,하계,"MAGGIE, NYAN, BRILLIANT MUSIC",2020,0.0,https://open.spotify.com/track/6Wtyb11rKgjVyDY...


In [105]:
streaming_2019 = top_2017_2021[top_2017_2021['year'] == 2019]

streaming_2019.loc[:,'song'] = streaming_2019.loc[:,'song'].str.replace('\"', '')
streaming_2019.loc[:,'song'] = streaming_2019.loc[:,'song'].str.upper()

streaming_2019.loc[:,'artist'] = streaming_2019.loc[:,'artist'].str.replace('\"', '')
streaming_2019.loc[:,'artist'] = streaming_2019.loc[:,'artist'].str.upper()
streaming_2019

,song,artist,year,streams,url
62401,RIDE IT,REGARD,2019,0.0,https://open.spotify.com/track/2tnVG71enUj33Ic...
62557,RANSOM,LIL TECCA,2019,0.0,https://open.spotify.com/track/6EOKwHETwSkZ9gW...
62715,IO E LEI,COMA,2019,0.0,https://open.spotify.com/track/0a1LpYLrp1VNwEe...
63033,VOICE,TAEYEON,2019,0.0,https://open.spotify.com/track/1esVK0wgkpFJ31e...
63086,SIREN SONG,MARUV,2019,0.0,https://open.spotify.com/track/1RgSs1Jy3QHiacs...
...,...,...,...,...,...
19330997,GET YOU THE MOON (FEAT. SNØW),KINA,2019,0.0,https://open.spotify.com/track/4ZLzoOkj0MPWrTL...
19330998,TRAMPOLINE,"SHAED, ZAYN",2019,0.0,https://open.spotify.com/track/4HBvwhhYo7YiYXk...
19330999,BLUEMING,IU,2019,0.0,https://open.spotify.com/track/4Dr2hJ3EnVh2Aao...
19331000,MY OH MY (FEAT. DABABY),CAMILA CABELLO,2019,0.0,https://open.spotify.com/track/3yOlyBJuViE2YSG...


In [107]:
sum_streaming_2019 = streaming_2019.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2019

,song,artist,year,streams,url
0,!,"SAMEY, GLEB",2019,421700.0,https://open.spotify.com/track/1A05ibu1DXGIt0F...
1,!,TRIPPIE REDD,2019,5077885.0,https://open.spotify.com/track/5a1ofLoPiAn5xxf...
2,# ENJOY YOUR LIFE,BLOUSON CHIEMI,2019,0.0,https://open.spotify.com/track/66TvvimCAB7DB07...
3,#24,DOLU KADEHI TERS TUT,2019,90529.0,https://open.spotify.com/track/6ihcJlZFxGvqIq7...
4,#302,KEN HIRAI,2019,591135.0,https://open.spotify.com/track/6XfTow9VT9OhotJ...
...,...,...,...,...,...
44690,두근거려 (BEAUTIFUL),BAEKHYUN,2019,0.0,https://open.spotify.com/track/7rxnGhTlqU3FRUz...
44691,믿어줄래,GOT7,2019,57825.0,https://open.spotify.com/track/3t9aeFQEtd2dhuT...
44692,첫사랑,EPITONE PROJECT,2019,0.0,https://open.spotify.com/track/7EBnhmiVuuIz3p4...
44693,첫사랑,SONDIA,2019,0.0,https://open.spotify.com/track/6ofS1EG2nH01GL5...


In [111]:
streaming_2018 = top_2017_2021[top_2017_2021['year'] == 2018]

streaming_2018.loc[:,'song'] = streaming_2018.loc[:,'song'].str.replace('\"', '')
streaming_2018.loc[:,'song'] = streaming_2018.loc[:,'song'].str.upper()

streaming_2018.loc[:,'artist'] = streaming_2018.loc[:,'artist'].str.replace('\"', '')
streaming_2018.loc[:,'artist'] = streaming_2018.loc[:,'artist'].str.upper()
streaming_2018

,song,artist,year,streams,url
1264,MINE,BAZZI,2018,62549.0,https://open.spotify.com/track/6tHWl8ows5JOZq9...
2390,OUTCAST,FOOL,2018,15936.0,https://open.spotify.com/track/6jnDfM8vtRWgN7R...
2545,SKYLDIG,BLAK,2018,10817.0,https://open.spotify.com/track/4k9zexSmoSS6EER...
3069,ENKELIT,CHEEK,2018,18974.0,https://open.spotify.com/track/6hAKGURpSwL2tvm...
4328,44 MORE,LOGIC,2018,1367372.0,https://open.spotify.com/track/6iyp6udVpLuq3MC...
...,...,...,...,...,...
25276069,OJALÁ (FEAT. DARELL),"DE LA GHETTO, ALMIGHTY, BRYANT MYERS",2018,1178.0,https://open.spotify.com/track/3EMDvnVpQd9RZJv...
25276070,LO QUE PASA EN LA NOCHE,MANO ARRIBA,2018,1178.0,https://open.spotify.com/track/2eOleVJlGvBE027...
25276071,EL EQUIVOCADO,MANO ARRIBA,2018,1170.0,https://open.spotify.com/track/5vy1C7DD9xJ5fRB...
25276072,QUE FUI TU AMANTE,EL GUCCI Y SU BANDA,2018,1165.0,https://open.spotify.com/track/1fmiCxwEbZFIszI...


In [112]:
sum_streaming_2018 = streaming_2018.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2018

,song,artist,year,streams,url
0,!,O.S.T.R.,2018,123714.0,https://open.spotify.com/track/5dwTUHjTcaMMkCD...
1,!I'LL BE BACK!,RILÈS,2018,36343.0,https://open.spotify.com/track/0fROT4kK5oTm8xO...
2,#AGORA ACABOU,"MC HENRY P, MC DANILO",2018,0.0,https://open.spotify.com/track/0cXSaSdx6Bhf6cA...
3,#B3,RK,2018,127295.0,https://open.spotify.com/track/7ivouaHYYikXDeh...
4,#B5,RK,2018,0.0,https://open.spotify.com/track/6pL93KyT7UhC48s...
...,...,...,...,...,...
48446,항상 (HANGSANG),"J-HOPE, SUPREME BOI",2018,141250.0,https://open.spotify.com/track/4mYu3kfBCW6qiTD...
48447,혼자 NOBODY KNOWS,GOT7,2018,6416.0,https://open.spotify.com/track/7HZsEJcqeDKTgzH...
48448,혼잣말 TALKING TO,DAY6,2018,0.0,https://open.spotify.com/track/5DzVHa9drSwpBRL...
48449,환상통 ONLY ONE YOU NEED,JONGHYUN,2018,0.0,https://open.spotify.com/track/51Men21ginqHFcS...


In [115]:
streaming_2017 = top_2017_2021[top_2017_2021['year'] == 2017]

streaming_2017.loc[:,'song'] = streaming_2017.loc[:,'song'].str.replace('\"', '')
streaming_2017.loc[:,'song'] = streaming_2017.loc[:,'song'].str.upper()

streaming_2017.loc[:,'artist'] = streaming_2017.loc[:,'artist'].str.replace('\"', '')
streaming_2017.loc[:,'artist'] = streaming_2017.loc[:,'artist'].str.upper()
streaming_2017

,song,artist,year,streams,url
0,CHANTAJE (FEAT. MALUMA),SHAKIRA,2017,253019.0,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
1,VENTE PA' CA (FEAT. MALUMA),RICKY MARTIN,2017,223988.0,https://open.spotify.com/track/7DM4BPaS7uofFul...
2,REGGAETÓN LENTO (BAILEMOS),CNCO,2017,210943.0,https://open.spotify.com/track/3AEZUABDXNtecAO...
3,SAFARI,"J BALVIN, PHARRELL WILLIAMS, BIA, SKY",2017,173865.0,https://open.spotify.com/track/6rQSrBHf7HlZjtc...
4,SHAKY SHAKY,DADDY YANKEE,2017,153956.0,https://open.spotify.com/track/58IL315gMSTD37D...
...,...,...,...,...,...
25992090,TWO GHOSTS,HARRY STYLES,2017,0.0,https://open.spotify.com/track/4B1rpPmQXwj78wk...
25992091,APOCALIPSIS ZOMBI,EL CUARTETO DE NOS,2017,0.0,https://open.spotify.com/track/063bMRonMLP3UEZ...
25992092,HEY THERE DELILAH,PLAIN WHITE T'S,2017,0.0,https://open.spotify.com/track/5O3eNPhckrOCxIB...
25992093,ME REHÚSO,DANNY OCEAN,2017,0.0,https://open.spotify.com/track/6Zosz8w6CmdGMZM...


In [116]:
sum_streaming_2017 = streaming_2017.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2017

,song,artist,year,streams,url
0,#0000FF,JASMINE SOKKO,2017,0.0,https://open.spotify.com/track/6dAROoyDvypjygW...
1,#23,DOLU KADEHI TERS TUT,2017,0.0,https://open.spotify.com/track/69LV8HvCI5vAMpB...
2,#9 - LIVE FROM HIGH SIERRA MUSIC FESTIVAL 2014,MOON HOOCH,2017,0.0,https://open.spotify.com/track/4eAJM4eDq4UYyDT...
3,#99,JVG,2017,31826.0,https://open.spotify.com/track/69lSnKz6dwvIxJ9...
4,#ASKIP,BLACK M,2017,296862.0,https://open.spotify.com/track/4T7fVWFjYNwkuCK...
...,...,...,...,...,...
55299,"태양의 후예 (ORIGINAL TELEVISION SOUNDTRACK), PT. 6...",K.WILL,2017,0.0,https://open.spotify.com/track/5bwZjteR12LZdlW...
55300,피 땀 눈물,BTS,2017,70997.0,https://open.spotify.com/track/1DyOhkfykPENmHb...
55301,함께 CURE,"TAEYONG, YOO YOUNG JIN",2017,0.0,https://open.spotify.com/track/2OTSH26Ohg9GPRX...
55302,혼자 하는 사랑 LONELY LOVE,SEOHYUN,2017,0.0,https://open.spotify.com/track/7d1BFNh1Cpg98vp...


In [101]:
merged_df = pd.merge(sum_streaming_2021, top_80s, on=['song', 'artist'], how='inner')
merged_df = merged_df.sort_values(by='Hot100 Rank')
merged_df

,song,artist,year,streams,url,Hot100 Rank
6,EYE OF THE TIGER,SURVIVOR,2021,11515.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...,4
4,EVERY BREATH YOU TAKE,THE POLICE,2021,267014286.0,https://open.spotify.com/track/1JSTJqkT5qHq8MD...,5
2,BILLIE JEAN,MICHAEL JACKSON,2021,21103413.0,https://open.spotify.com/track/5ChkMS8OtdzJeqy...,17
9,JESSIE'S GIRL,RICK SPRINGFIELD,2021,45615.0,https://open.spotify.com/track/5RsUlxLto4NZbhJ...,35
3,DOWN UNDER,MEN AT WORK,2021,1031618.0,https://open.spotify.com/track/5pSvjjfsh34sLrk...,39
8,I WANNA DANCE WITH SOMEBODY (WHO LOVES ME),WHITNEY HOUSTON,2021,28893046.0,https://open.spotify.com/track/2tUBqZG2AbRi7Q0...,50
10,LIVIN' ON A PRAYER,BON JOVI,2021,219754.0,https://open.spotify.com/track/37ZJ0p5Jm13JPev...,52
1,BEAT IT,MICHAEL JACKSON,2021,143820.0,https://open.spotify.com/track/1OOtq8tRnDM8kG2...,55
5,EVERYBODY WANTS TO RULE THE WORLD,TEARS FOR FEARS,2021,11044653.0,https://open.spotify.com/track/4RvWPyQ5RL0ao9L...,67
7,GHOSTBUSTERS,RAY PARKER JR.,2021,6670088.0,https://open.spotify.com/track/6thXB4RmajS4oZP...,78


In [102]:
merged_df_2 = pd.merge(sum_streaming_2020, top_80s, on=['song', 'artist'], how='inner')
merged_df_2 = merged_df_2.sort_values(by='Hot100 Rank')
merged_df_2

,song,artist,year,streams,url,Hot100 Rank
1,BETTE DAVIS EYES,KIM CARNES,2020,0.0,https://open.spotify.com/track/0odIT9B9BvOCnXf...,2
6,EYE OF THE TIGER,SURVIVOR,2020,0.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...,4
4,EVERY BREATH YOU TAKE,THE POLICE,2020,65291759.0,https://open.spotify.com/track/1JSTJqkT5qHq8MD...,5
11,LADY,KENNY ROGERS,2020,15298.0,https://open.spotify.com/track/5lllSpYR5kRirQk...,10
2,BILLIE JEAN,MICHAEL JACKSON,2020,1441841.0,https://open.spotify.com/track/5ChkMS8OtdzJeqy...,17
15,"SAY YOU, SAY ME",LIONEL RICHIE,2020,0.0,https://open.spotify.com/track/17CPezzLWzvGfpZ...,19
18,TOTAL ECLIPSE OF THE HEART,BONNIE TYLER,2020,0.0,https://open.spotify.com/track/7wuJGgpTNzbUyn2...,31
10,JESSIE'S GIRL,RICK SPRINGFIELD,2020,144880.0,https://open.spotify.com/track/5RsUlxLto4NZbhJ...,35
3,DOWN UNDER,MEN AT WORK,2020,524941.0,https://open.spotify.com/track/5pSvjjfsh34sLrk...,39
9,I WANNA DANCE WITH SOMEBODY (WHO LOVES ME),WHITNEY HOUSTON,2020,20743144.0,https://open.spotify.com/track/2tUBqZG2AbRi7Q0...,50


In [108]:
merged_df_3 = pd.merge(sum_streaming_2019, top_80s, on=['song', 'artist'], how='inner')
merged_df_3 = merged_df_3.sort_values(by='Hot100 Rank')
merged_df_3

,song,artist,year,streams,url,Hot100 Rank
10,EYE OF THE TIGER,SURVIVOR,2019,92970.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...,4
8,EVERY BREATH YOU TAKE,THE POLICE,2019,56106645.0,https://open.spotify.com/track/1JSTJqkT5qHq8MD...,5
5,CALL ME,BLONDIE,2019,0.0,https://open.spotify.com/track/4qO03RMQm88DdpT...,9
3,BILLIE JEAN,MICHAEL JACKSON,2019,1767630.0,https://open.spotify.com/track/5ChkMS8OtdzJeqy...,17
25,"SAY YOU, SAY ME",LIONEL RICHIE,2019,0.0,https://open.spotify.com/track/17CPezzLWzvGfpZ...,19
30,WHEN DOVES CRY,PRINCE,2019,0.0,https://open.spotify.com/track/51H2y6YrNNXcy3d...,26
27,TOTAL ECLIPSE OF THE HEART,BONNIE TYLER,2019,97089.0,https://open.spotify.com/track/4McRlwqJQIERlJF...,31
6,DON'T YOU WANT ME,THE HUMAN LEAGUE,2019,52431.0,https://open.spotify.com/track/3L7RtEcu1Hw3OXr...,34
17,JESSIE'S GIRL,RICK SPRINGFIELD,2019,1087341.0,https://open.spotify.com/track/5RsUlxLto4NZbhJ...,35
7,DOWN UNDER,MEN AT WORK,2019,682450.0,https://open.spotify.com/track/7jTKLKs3NDmDnJH...,39


In [113]:
merged_df_4 = pd.merge(sum_streaming_2018, top_80s, on=['song', 'artist'], how='inner')
merged_df_4 = merged_df_4.sort_values(by='Hot100 Rank')
merged_df_4

,song,artist,year,streams,url,Hot100 Rank
3,BETTE DAVIS EYES,KIM CARNES,2018,48560.0,https://open.spotify.com/track/0odIT9B9BvOCnXf...,2
11,EYE OF THE TIGER,SURVIVOR,2018,116877.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...,4
5,CALL ME,BLONDIE,2018,0.0,https://open.spotify.com/track/4qO03RMQm88DdpT...,9
4,BILLIE JEAN,MICHAEL JACKSON,2018,10657045.0,https://open.spotify.com/track/5ChkMS8OtdzJeqy...,17
29,"SAY YOU, SAY ME",LIONEL RICHIE,2018,0.0,https://open.spotify.com/track/0I9M3okWVRJXWT6...,19
7,DO THAT TO ME ONE MORE TIME,CAPTAIN & TENNILLE,2018,0.0,https://open.spotify.com/track/5HoBwCGMxi2TY1f...,24
21,LIKE A VIRGIN,MADONNA,2018,0.0,https://open.spotify.com/track/1ZPlNanZsJSPK5h...,25
31,TOTAL ECLIPSE OF THE HEART,BONNIE TYLER,2018,107576.0,https://open.spotify.com/track/4McRlwqJQIERlJF...,31
8,DON'T YOU WANT ME,THE HUMAN LEAGUE,2018,57964.0,https://open.spotify.com/track/3L7RtEcu1Hw3OXr...,34
18,JESSIE'S GIRL,RICK SPRINGFIELD,2018,153711.0,https://open.spotify.com/track/5RsUlxLto4NZbhJ...,35


In [117]:
merged_df_5 = pd.merge(sum_streaming_2017, top_80s, on=['song', 'artist'], how='inner')
merged_df_5 = merged_df_4.sort_values(by='Hot100 Rank')
merged_df_5

,song,artist,year,streams,url,Hot100 Rank
3,BETTE DAVIS EYES,KIM CARNES,2018,48560.0,https://open.spotify.com/track/0odIT9B9BvOCnXf...,2
11,EYE OF THE TIGER,SURVIVOR,2018,116877.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...,4
5,CALL ME,BLONDIE,2018,0.0,https://open.spotify.com/track/4qO03RMQm88DdpT...,9
4,BILLIE JEAN,MICHAEL JACKSON,2018,10657045.0,https://open.spotify.com/track/5ChkMS8OtdzJeqy...,17
29,"SAY YOU, SAY ME",LIONEL RICHIE,2018,0.0,https://open.spotify.com/track/0I9M3okWVRJXWT6...,19
7,DO THAT TO ME ONE MORE TIME,CAPTAIN & TENNILLE,2018,0.0,https://open.spotify.com/track/5HoBwCGMxi2TY1f...,24
21,LIKE A VIRGIN,MADONNA,2018,0.0,https://open.spotify.com/track/1ZPlNanZsJSPK5h...,25
31,TOTAL ECLIPSE OF THE HEART,BONNIE TYLER,2018,107576.0,https://open.spotify.com/track/4McRlwqJQIERlJF...,31
8,DON'T YOU WANT ME,THE HUMAN LEAGUE,2018,57964.0,https://open.spotify.com/track/3L7RtEcu1Hw3OXr...,34
18,JESSIE'S GIRL,RICK SPRINGFIELD,2018,153711.0,https://open.spotify.com/track/5RsUlxLto4NZbhJ...,35


In [119]:
final_df = pd.concat([merged_df, merged_df_2, merged_df_3, merged_df_4, merged_df_5], axis=0)
final_df = final_df.sort_values(by='streams', ascending=False)
final_df = final_df.drop_duplicates(subset=['song', 'artist'], keep='first')
final_df = final_df.sort_values(by='Hot100 Rank')
print(len(final_df))
final_df

41


,song,artist,year,streams,url,Hot100 Rank
3,BETTE DAVIS EYES,KIM CARNES,2018,48560.0,https://open.spotify.com/track/0odIT9B9BvOCnXf...,2
11,EYE OF THE TIGER,SURVIVOR,2018,116877.0,https://open.spotify.com/track/2HHtWyy5CgaQbC7...,4
4,EVERY BREATH YOU TAKE,THE POLICE,2021,267014286.0,https://open.spotify.com/track/1JSTJqkT5qHq8MD...,5
5,CALL ME,BLONDIE,2018,0.0,https://open.spotify.com/track/4qO03RMQm88DdpT...,9
11,LADY,KENNY ROGERS,2020,15298.0,https://open.spotify.com/track/5lllSpYR5kRirQk...,10
2,BILLIE JEAN,MICHAEL JACKSON,2021,21103413.0,https://open.spotify.com/track/5ChkMS8OtdzJeqy...,17
29,"SAY YOU, SAY ME",LIONEL RICHIE,2018,0.0,https://open.spotify.com/track/0I9M3okWVRJXWT6...,19
7,DO THAT TO ME ONE MORE TIME,CAPTAIN & TENNILLE,2018,0.0,https://open.spotify.com/track/5HoBwCGMxi2TY1f...,24
21,LIKE A VIRGIN,MADONNA,2018,0.0,https://open.spotify.com/track/1ZPlNanZsJSPK5h...,25
30,WHEN DOVES CRY,PRINCE,2019,0.0,https://open.spotify.com/track/51H2y6YrNNXcy3d...,26
